In [1]:
import sys
import dgl
import dgl.function as fn
import os
import multiprocessing as mp
from tqdm import tqdm
import pdb
import numpy as np
import torch
import torch.nn as nn
import logging
from utils.parser import parse_args
from utils.dataloader import Dataloader
from utils.utils import config, construct_negative_graph, choose_model, load_mf_model, NegativeGraph
from utils.tester import Tester
from models.sampler import NegativeSampler
import wandb

In [2]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', default = 'TaoBao', type = str,
                    help = 'Dataset to use')
parser.add_argument('--seed', default = 2022, type = int,
                    help = 'seed for experiment')
parser.add_argument('--embed_size', default = 32, type = int,
                    help = 'embedding size for all layer')
parser.add_argument('--lr', default = 0.05, type = float,
                    help = 'learning rate')
parser.add_argument('--weight_decay', default = 8e-8, type = float,
                    help = "weight decay for adam optimizer")
#######################################################################
parser.add_argument('--model', default ='test',type = str,
                    help = 'model selection')#dgrec base moe test
parser.add_argument('--epoch', default = 1000, type = int,
                    help = 'epoch number')
parser.add_argument('--patience', default = 10, type = int,
                    help = 'early_stop validation')
parser.add_argument('--batch_size', default = 2048, type = int,
                    help = 'batch size')
parser.add_argument('--layers', default = 1, type = int,
                    help = 'layer number')
parser.add_argument('--gpu', default = 0, type = int,
                    help = '-1 for cpu, 0 for gpu:0')
parser.add_argument('--k_list', default = [100, 300], type = list,
                    help = 'topk evaluation')
parser.add_argument('--k', default = 20, type = int,
                    help = 'neighbor number in each GNN aggregation')
parser.add_argument('--neg_number', default = 4, type = int,
                    help = 'negative sampler number for each positive pair')
parser.add_argument('--metrics', default = ['recall', 'hit_ratio', 'coverage'])


parser.add_argument('--sigma', default = 1.0, type = float,
                    help = 'sigma for gaussian kernel')
parser.add_argument('--gamma', default = 2.0, type = float,
                    help = 'gamma for gaussian kernel')
################################################################################
parser.add_argument('--category_balance', default = True, type = bool,
                    help = 'whether make loss category balance')
parser.add_argument('--beta_class', default = 0.9, type = float,
                    help = 'class re-balanced loss beta')
parser.add_argument('--context_code_dim', default = 32, type = int,
                    help = 'interest num')
parser.add_argument('--num_context_codes', default = 32, type = int,
                    help = 'interest dim')
parser.add_argument('--n_experts', default = 5, type = int,
                    help = 'n_experts')
##########################################################################################
parser.add_argument('--wandb_enable', default = True, type = bool,
                    help = 'layer number')
parser.add_argument('--hidden_size', default = 32, type = int,
                        help = 'n_experts')
parser.add_argument('--k_experts', default = 2, type = int,
                            help = 'n_experts')
parser.add_argument('--moe', default = False, type = bool,
                            help = 'layer number')
################################################################################
parser.add_argument('--sub', default = 'rand', type = str,
                            help = 'layer number')

args = parser.parse_args([])

In [3]:
data = args.dataset
if args.gpu >= 0 and torch.cuda.is_available():
        device = 'cuda:{}'.format(args.gpu)
else:
        device = 'cpu'
device = torch.device(device)
args.device = device
dataloader = Dataloader(args, data, device)
graph = dataloader.train_graph

hid_dim = args.embed_size
user_embedding = torch.nn.Parameter(torch.randn(graph.nodes('user').shape[0], hid_dim))
item_embedding = torch.nn.Parameter(torch.randn(graph.nodes('item').shape[0], hid_dim))
node_features = {'user': user_embedding, 'item': item_embedding}
etype=('item', 'rated by', 'user')




100%|██████████| 136710/136710 [00:00<00:00, 4272210.67it/s]


In [8]:
print(device)

cuda:0


In [92]:
graph = dataloader.train_graph
feat_src = node_features['item'].to(device)
graph.nodes['item'].data['h'] = feat_src
cata_num=dataloader.category_num
cata_embedding = torch.nn.Parameter(torch.randn(cata_num, 32)).to(device)

In [144]:
import math
import random
maxlen=20
def cate_topsis(numbers):
        square_sum=math.sqrt(sum(i**2 for i in numbers))
        numbers = [math.exp(x/square_sum) for x in numbers]
        ma=max(numbers)
        mi=0.5*min(numbers)
        result=[(x-mi) /(ma-mi) for x in numbers]
        result = [x / sum(result) for x in result]
        result=[math.ceil(x*maxlen) for x in result]
        return result
def pairwise_cosine_similarity(x, y, zero_diagonal: bool = False):
    r"""
    Calculates the pairwise cosine similarity matrix

    Args:
        x: tensor of shape ``(batch_size, M, d)``
        y: tensor of shape ``(batch_size, N, d)``
        zero_diagonal: determines if the diagonal of the distance matrix should be set to zero

    Returns:
        A tensor of shape ``(batch_size, M, N)``
    """
    x_norm = torch.linalg.norm(x, dim=1, keepdim=True)
    y_norm = torch.linalg.norm(y, dim=1, keepdim=True)
    distance = torch.matmul(torch.div(x, x_norm), torch.div(y, y_norm).permute(1, 0))
    

    return distance
def submodular_selection_feature(nodes):
        device = nodes.mailbox['m'].device
        mail = nodes.mailbox['m']
        batch_size, neighbor_size, feature_size = mail.shape
        cat=nodes.mailbox['c']
        user_select=[]
        
        
        for i in range(batch_size):
            select=[]
            line=cat[i].reshape(1,-1)[0].tolist()
            unique_elements = list(set(line))
            unique_cat=cata_embedding[unique_elements]
            sim=pairwise_cosine_similarity(unique_cat,unique_cat).mean(dim=1)
            sim=sim.tolist()
            max_sim=round(max(sim),2)
            min_sim=round(min(sim),2)
            if max_sim==min_sim:
                 max_sim=round(2*max_sim,2)
            moth=round(max_sim-min_sim,2)
            element_indices = {}#1111111
           
            for index, element in enumerate(line):
                if element in element_indices:
                   element_indices[element].append(index)
                else:
                   element_indices[element] = [index]
            element_counts = [line.count(element) for element in unique_elements]
            sorted_indices = sorted(range(len(element_counts)), key=lambda i: element_counts[i], reverse=True)
            for i in sorted_indices:
               my_list=element_indices[unique_elements[i]]
               random_elements=random.choices(my_list, k=math.ceil(0.8*round((max_sim-sim[i])/(moth),2)*len(my_list)))
               
               select=select+random_elements
               if len(select)>=maxlen:
                   break   
            if len(select)>=maxlen:
                select=select[0:maxlen]
            else:
                while len(select)<maxlen:
                      select=select+select+select
                select=select[0:maxlen]      
           
            user_select.append(select)
        user_select=torch.tensor(user_select)
        
        return user_select

In [103]:

import math

def category_aggregation(edges):
        return {'c': edges.src['category'], 'm': edges.src['h']}
def sub_reduction_item_user(nodes):
        # -1 indicate user-> node, which does not include category information
        device = nodes.mailbox['m'].device
        mail = nodes.mailbox['m']
        cat=nodes.mailbox['c']
        
        batch_size, neighbor_size, feature_size = mail.shape
        if (-1 in nodes.mailbox['c']) or nodes.mailbox['m'].shape[1] <=20:
            mail=mail.sum(dim=1)
        else:
            neighbors =submodular_selection_feature(nodes)
            mail = mail[torch.arange(batch_size, dtype = torch.long, device = mail.device).unsqueeze(-1), neighbors]
            mail=mail.sum(dim=1)

       
    
        return {'h': mail}

In [145]:
with graph.local_scope():
    graph.update_all(category_aggregation, sub_reduction_item_user, etype = etype)

In [7]:
x=torch.tensor([[1,3,4],[3,4,5]])
cata_embedding[x]

tensor([[[ 1.4066e+00, -1.3300e+00,  2.0119e+00, -2.0197e+00, -1.0050e+00,
           3.2687e-01,  4.3971e-01, -1.5703e+00,  1.2183e+00, -4.9825e-01,
          -2.7870e-01, -4.0875e-01, -7.0351e-01, -1.2346e-01,  1.0357e+00,
          -1.5857e+00,  6.0278e-01,  2.8883e+00, -1.0026e-01,  1.0496e-01,
          -1.5525e+00, -3.4744e-01, -6.6411e-02, -7.5969e-01, -2.6731e-01,
          -7.0714e-01, -8.9049e-01, -1.0908e+00, -1.0498e+00,  1.0004e+00,
           6.5227e-01,  1.5745e+00],
         [-1.7382e+00, -7.9115e-03, -3.3640e-02,  8.7213e-01, -9.3560e-01,
          -8.7332e-01,  1.4803e+00, -1.3726e+00,  7.8083e-03, -1.3623e+00,
          -2.8310e-01, -5.0738e-01, -1.3752e+00, -1.2206e+00, -3.6263e-01,
           8.5177e-01,  1.8989e-01, -1.0751e+00, -7.7584e-02,  2.3632e-03,
          -9.4927e-02, -4.8747e-02, -2.2387e-01,  6.4526e-02, -4.7183e-01,
          -8.6214e-01,  5.1349e-01,  1.1047e+00,  5.9435e-01,  1.8810e+00,
           1.4664e+00,  4.0705e-01],
         [-2.1008e+00, -7.